# Plain

In [2]:
import copy
import json
import numpy as np
import pandas as pd
import pydeck as pdk
import pycountry
import streamlit as st
import streamlit.components.v1 as components

OG_DATA_URL = "./maps/all-countries.geojson"
UN_URL = './maps/un_countries.geojson'
CENTROIDS = './maps/centroids.geojson'
gdp_un = pd.read_csv("./data/gdp-per-capita-un.csv")
migrant_workers = pd.read_csv("./data/migrant-workers-un.csv")
migrant_workers = migrant_workers[migrant_workers['sex.label'] == 'Sex: Total']
migrant_workers = migrant_workers[migrant_workers['classif1.label'] == 'Age (Youth, adults): 15-24']
migrant_workers = migrant_workers[['ref_area.label', 'time', 'obs_value']]
migrant_workers.dropna(inplace=True)
gdp_un.dropna(inplace=True)

alpha3_numeric_name = [(country.alpha_3, country.numeric, country.name) for country in pycountry.countries]

migrant_workers['highest_of_the_year'] = migrant_workers.apply(lambda row: np.log(np.max(migrant_workers[migrant_workers['time'] == row['time']]['obs_value'].values.tolist())), axis=1)

migrant_workers['obs_value'] = migrant_workers['obs_value'].apply(lambda x: np.log(x)) / migrant_workers['highest_of_the_year'] * 255

not_exist = 0
exist = 0
un_countries = []
with open(OG_DATA_URL) as f:
    arr = json.load(f)

    for i, country in enumerate(arr['features']):
        alpha3 = country['properties']['shapeGroup']
        if alpha3 in list(np.array(alpha3_numeric_name)[:, 0]):
            numeric_index = list(np.array(alpha3_numeric_name)[:, 0]).index(alpha3)
            numeric = alpha3_numeric_name[numeric_index][1]
            name = alpha3_numeric_name[numeric_index][2]

            if int(numeric) in gdp_un['Country or Area Code'].to_numpy().reshape(-1, ):
                d = {str(int(key)): value for key, value in
                     gdp_un[gdp_un['Country or Area Code'] == int(numeric)][['Year', 'Value']].values.tolist()}
                country['properties']['gdpc'] = d

                d_m = {str(int(key)): value for key, value in
                       migrant_workers[migrant_workers['ref_area.label'] == name][
                           ['time', 'obs_value']].values.tolist()}
                country['properties']['youth_workers'] = d_m

                un_countries.append(country)
        else:
            not_exist += 1
            continue

    with open(UN_URL, 'w') as outfile:
        arr_copy = copy.deepcopy(arr)

        # Get top 50 countries by GDP per capita in 2020
        top_50 = sorted(
            un_countries,
            key=lambda d: d['properties'].get('gdpc', {}).get('2020', 0),
            reverse=True
        )

        # Reduce coordinates for each country
        for i, country in enumerate(top_50):
            geometry_type = country['geometry']['type']
            if geometry_type == "Polygon":
                # Simplify single polygon
                for ring in country['geometry']['coordinates']:
                    if len(ring) > 100:  # Reduce only if there are enough points
                        ring[:] = ring[::100] + [ring[-1]]
            elif geometry_type == "MultiPolygon":
                # Simplify multipolygon
                for polygon in country['geometry']['coordinates']:
                    for ring in polygon:
                        if len(ring) > 100:
                            ring[:] = ring[::100] + [ring[-1]]

        # Update the feature collection with reduced features
        arr_copy['features'] = top_50

        json.dump(arr_copy, outfile)

type
crs
  type
  properties
    name
features


# Log-scale

In [4]:
OG_DATA_URL = "./maps/all-countries.geojson"
UN_URL = './maps/un_countries_log.geojson'
CENTROIDS = './maps/centroids.geojson'
gdp_un = pd.read_csv("./data/gdp-per-capita-un.csv")
migrant_workers = pd.read_csv("./data/migrant-workers-un.csv")
migrant_workers = migrant_workers[migrant_workers['sex.label'] == 'Sex: Total']
migrant_workers = migrant_workers[migrant_workers['classif1.label'] == 'Age (Youth, adults): 15-24']
migrant_workers = migrant_workers[['ref_area.label', 'time', 'obs_value']]
migrant_workers.dropna(inplace=True)
gdp_un.dropna(inplace=True)

alpha3_numeric_name = [(country.alpha_3, country.numeric, country.name) for country in pycountry.countries]

migrant_workers['highest_of_the_year'] = migrant_workers.apply(lambda row: np.log(np.max(migrant_workers[migrant_workers['time'] == row['time']]['obs_value'].values.tolist())), axis=1)

migrant_workers['obs_value'] = migrant_workers['obs_value'].apply(lambda x: np.log(x)) / migrant_workers['highest_of_the_year'] * 255

not_exist = 0
exist = 0
un_countries = []
with open(OG_DATA_URL) as f:
    arr = json.load(f)

    for i, country in enumerate(arr['features']):
        alpha3 = country['properties']['shapeGroup']
        if alpha3 in list(np.array(alpha3_numeric_name)[:, 0]):
            numeric_index = list(np.array(alpha3_numeric_name)[:, 0]).index(alpha3)
            numeric = alpha3_numeric_name[numeric_index][1]
            name = alpha3_numeric_name[numeric_index][2]

            if int(numeric) in gdp_un['Country or Area Code'].to_numpy().reshape(-1, ):
                d = {str(int(key)): np.log(value).item() for key, value in
                     gdp_un[gdp_un['Country or Area Code'] == int(numeric)][['Year', 'Value']].values.tolist()}
                country['properties']['gdpc'] = d

                d_m = {str(int(key)): value for key, value in
                       migrant_workers[migrant_workers['ref_area.label'] == name][
                           ['time', 'obs_value']].values.tolist()}
                country['properties']['youth_workers'] = d_m

                un_countries.append(country)
        else:
            not_exist += 1
            continue

    with open(UN_URL, 'w') as outfile:
        arr_copy = copy.deepcopy(arr)

        # Get top 50 countries by GDP per capita in 2020
        top_50 = sorted(
            un_countries,
            key=lambda d: d['properties'].get('gdpc', {}).get('2020', 0),
            reverse=True
        )

        # Reduce coordinates for each country
        for i, country in enumerate(top_50):
            geometry_type = country['geometry']['type']
            if geometry_type == "Polygon":
                # Simplify single polygon
                for ring in country['geometry']['coordinates']:
                    if len(ring) > 100:  # Reduce only if there are enough points
                        ring[:] = ring[::100] + [ring[-1]]
            elif geometry_type == "MultiPolygon":
                # Simplify multipolygon
                for polygon in country['geometry']['coordinates']:
                    for ring in polygon:
                        if len(ring) > 100:
                            ring[:] = ring[::100] + [ring[-1]]

        # Update the feature collection with reduced features
        arr_copy['features'] = top_50

        json.dump(arr_copy, outfile)